지하철 유무임별 이용현황 데이터 정제하기

In [1]:
import csv
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

In [2]:
# 한글 깨짐 방지
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

plt.rc('font', family = 'NanumBarunGothic') # 나눔 바른 고딕을 기본 글꼴로 설정
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
# next(data)

for row in data:
  print(row)

In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

for row in data:
  row[4] = int(row[4])
  print(row)

인원 숫자 데이터 정수로 변경 (유임승차 유임하차 무임승차 무임하차 :4,5,6,7) 인덱스

In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

for row in data:
  for i in range(4,8):    # 4, 5, 6, 7 인덱스 데이터 정수로 변경
    row[i] = int(row[i])
  print(row)

유임 승차 비율이 가장 높은 역은 어디일까

유임승차 / 무임승차 비율의 최댓값 찾기 -> 기존 0 데이터가 '-' 데이터로 되어 있음

rate = 유임승차 인원 / 무임 승차 인원

In [16]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = 0   # 최대 유임승차 비율
rate = 0  # 유임승차 비율

for row in data:
  for i in range(4,8):    # 4, 5, 6, 7 인덱스 데이터 정수로 변경
    row[i] = int(row[i])
  
  if row[4] != 0 and row[6] != 0:
    rate = row[4] / row[6]  # 유임승차 / 무임승차
  
  if rate > max:
    max = rate

print(max)

27.5


비율 계산 방식 변경 rate = 유임승차 인원 / 전체(유임 + 무임) 인원


In [21]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = 0   # 최대 유임승차 비율
rate = 0  # 유임승차 비율

for row in data:
  for i in range(4,8):    # 4, 5, 6, 7 인덱스 데이터 정수로 변경
    row[i] = int(row[i])
  
  if row[4] != 0 and row[6] != 0:
    rate = row[4] / (row[4] + row[6])  # 유임승차 / 전체(유임 + 무임)
  
  if rate > max:
    max = rate
    max_station = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex) 서울역 1호선
    print(max_station, round(max * 100, 2))

서울역 1호선 87.27
종각 1호선 88.43
시청 2호선 92.04
을지로입구 2호선 92.16
한양대 2호선 93.83
강남 2호선 94.42
홍대입구 2호선 95.34
충무로 3호선 96.49


너무 작은 데이터 거르기

In [23]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = 0   # 최대 유임승차 비율
rate = 0  # 유임승차 비율
max_station = ''  # 최대 유임승차 역 정보

for row in data:
  for i in range(4,8):    # 4, 5, 6, 7 인덱스 데이터 정수로 변경
    row[i] = int(row[i])
  
  if row[4] != 0 and row[6] != 0 and (row[4] + row[6]) > 100000:    # 데이터가 너무 적은 곳은 쓰지 말자하
    rate = row[4] / (row[4] + row[6])  # 유임승차 / 전체(유임 + 무임)
  
  if rate > max:
    max = rate
    max_station = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex) 서울역 1호선

print(max_station, round(max * 100, 2))

홍대입구 2호선 95.34


유무임 승하차 인원이 가장 많은 역 찾기

In [26]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = [0] * 4   # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수
max_station = [''] * 4  # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] > max[i-4]:                       # row[4] > max[0] 유임승차 비교, row[5] > max[1] 유임하차 비교......
      max[i-4] = row[i]
      max_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + max_station[i], max[i])

유임승차 강남 2호선 3153418
유임하차 강남 2호선 3210437
무임승차 종로3가 1호선 387062
무임하차 제기동 1호선 400607


유무임 승하차 인원이 가장 적은 역 찾기

In [37]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

min = [999] * 4   # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수
min_station = [''] * 4  # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] < min[i-4] and row[i] != 0:                       # row[4] < max[0] 유임승차 비교, row[5] < max[1] 유임하차 비교......
      min[i-4] = row[i]
      min_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + min_station[i], min[i])

유임승차 지축 일산선 2
유임하차 지평 중앙선 566
무임승차 충무로 3호선 2
무임하차 옥수 경원선 91


In [35]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

min = [999] * 4   # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수
min_station = [''] * 4  # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] < min[i-4]:                       # row[4] < max[0] 유임승차 비교, row[5] < max[1] 유임하차 비교......
      min[i-4] = row[i]
      min_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + min_station[i], min[i])

유임승차 지축 일산선 2
유임하차 충무로 3호선 0
무임승차 창동 경원선 0
무임하차 충무로 3호선 0


In [39]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

min = [999] * 4   # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수
min_station = [''] * 4  # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] <= min[i-4] and row[i] != 0:                       # row[4] < max[0] 유임승차 비교, row[5] < max[1] 유임하차 비교......
      min[i-4] = row[i]
      min_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + min_station[i], min[i])

유임승차 홍대입구 경의선 2
유임하차 지평 중앙선 566
무임승차 연신내 6호선 2
무임하차 옥수 경원선 91


In [40]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

min = [999] * 4   # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수
min_station = [''] * 4  # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] <= min[i-4]:                       # row[4] < max[0] 유임승차 비교, row[5] < max[1] 유임하차 비교......
      min[i-4] = row[i]
      min_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + min_station[i], min[i])

유임승차 홍대입구 경의선 2
유임하차 연신내 6호선 0
무임승차 검암 경의선 0
무임하차 연신내 6호선 0


In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

min = [999] * 4   # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수
min_station = [''] * 4  # 최소 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i]) 
    if row[i] < min[i-4] and row[i] != 0:                       # row[4] < max[0] 유임승차 비교, row[5] < max[1] 유임하차 비교......
      min[i-4] = row[i]
      min_station[i-4] = row[3] + ' ' + row[1]  # 지하철 역 + 호선 정보 ex)서울역 1호선

for i in range(4):
  print(label[i] + ' ' + min_station[i], min[i])

모든 역의 유무임 승하차 비율은 어떻게 될까?

In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = [0] * 4   # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수
max_station = [''] * 4  # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i])
    plt.figure(dpi = 120)
    plt.pie(row[4:8])
    plt.axis('equal')
    plt.show()

모든 역의 유무임 승하차 비율을 파이 차트로 나타내기

In [ ]:
f = open('/content/subwayfee.csv') # 인코딩이 utf-8 이여서 디폴트값으로 생략 가능
data = csv.reader(f)
next(data)

max = [0] * 4   # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수
max_station = [''] * 4  # 최대 유임승차, 유임하차, 무임승차, 무임하차 인원 수 역 정보

label = ['유임승차', '유임하차', '무임승차', '무임하차']

for row in data:
  for i in range(4,8):
    row[i] = int(row[i])
    plt.figure(dpi = 120)
    plt.title(row[3] + ' ' + row[1])
    plt.pie(row[4:8], labels = label, autopct = '%1.f%%')
    plt.axis('equal')
    plt.savefig(row[3] + ' ' + row[1] + '.png')   # 함부로 실행시키지 마라
    plt.show()